## Protecting Life Data Analysis Script

- Version 1.2.1 - NHS Pycom Version Built - 22/05/2022
- Version 1.1.3 - Current Version Demo delivered to Divisional Management - 14/04/2022
- Version 1.1.2 - Abstract Submitted to BSG Conference 2022 - 25/02/2022
- Version 1.1.1 - Basic MVP Built - 23/02/2022

#### Authors:

1. Matt Stammers - Consultant Gastroenterolgist and Data Scientist @ AXIS, UHS
2. Michael George - Data Engineering Lead @ AXIS, UHS

What this Script Does:
- Loads in the data prepared by the data preparation script
- Performs mass statistical descripitve analysis on the data after dividing the cohorts into two groups
- Performs further statistical analysis on the data

#### The first stage in an analytics package is to load in the analytics packages - ideally keeping this a slim as possible

We have tried to use only a fairly simple selection of packages in this analytics pipeline - this then makes it much easier to build upon later on. Where possible we have coded out statistical functions ourselves to make the code even easier to understand

In [1]:
# Import Key Packages

# Import base packages
import math
from datetime import datetime
from datetime import timedelta

# Import data manipulation packages
import numpy as np
import pandas as pd

# Import Statistical Packages. We are using Chi2 and Mann Whitney U Test primary here

from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats import mannwhitneyu
import statsmodels.api as sm

# Import Plotting Packages

import matplotlib.pyplot as plt
import seaborn as sns

#### Pandas settings adjustment

Typically when performing analytics I like to adjust the base pandas settings for maximal customisability. This is up to you but if you want to change the settings you can below in any way you wish

In [2]:
# Adjust settings to see entire frame:

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('mode.chained_assignment', 'warn')

#### Below is the funciton block. Here we define our own user-generated functions to help with learning

We have written these out bespoke. You can just use the ones contained in other packages but for the sake of learning and clarity we have written out the functions bespoke unless they are contained as defaults within numpy and pandas in which case they are highly reliable. If you spot a mistake please let us know!

In [3]:
# Statistical Functions

def IQR(x):
    """Computes Interquartile Range"""
    lb = stats.scoreatpercentile(x, 25) # Calculates 25th percentile
    ub = stats.scoreatpercentile(x, 75) # Calculates 75th percentile
    return ub - lb # Subtracts one from the other

def IQR_lower(x):
    """Computes Interquartile Range Lower Quartile"""
    lb = stats.scoreatpercentile(x, 25) # Calculates 25th percentile
    return lb

def IQR_upper(x):
    """Computes Interquartile Range Upper Quartile"""
    ub = stats.scoreatpercentile(x, 75) # Calculates 75th percentile
    return ub

def std_lower(x):
    """Computes Lower Standard Deviation of the Mean"""
    s = np.std(x) # Calculates Standard Deviation
    m = np.mean(x) # Calculates Mean
    return  round(m-s,4)

def std_upper(x):
    """Computes Upper Standard Deviation of the Mean"""
    s = np.std(x) # Calculates Standard Deviation
    m = np.mean(x) # Calculates Mean
    return  round(m+s,4)

def CI_lower(x):
    """Computes 95% Confidence Interval Lower Bound of the Mean"""
    alpha = 0.05                       # significance level = 5%
    degfree = len(x) - 1                  # degress of freedom
    t = stats.t.ppf(1 - alpha/2, degfree)   # 95% confidence t-score 
    s = np.std(x)          # sample standard deviation 
    n = len(x)
    m = np.mean(x)
    return  round(m - (t * s / np.sqrt(n)),4)

def CI_upper(x):
    """Computes 95% Confidence Interval Upper Bound of the Mean"""
    alpha = 0.05                       # significance level = 5%
    degfree = len(x) - 1                  # degress of freedom
    t = stats.t.ppf(1 - alpha/2, degfree)   # 95% confidence t-score 
    s = np.std(x)          # sample standard deviation 
    n = len(x)
    m = np.mean(x)
    return  round(m + (t * s / np.sqrt(n)),4)

def CI_lower_median(x):
    """Computes 95% Confidence Interval Lower Bound of the Median"""
    alpha = 0.05                       # significance level = 5%
    degfree = len(x) - 1                  # degress of freedom
    t = stats.t.ppf(1 - alpha/2, degfree)   # 95% confidence t-score 
    s = np.std(x)          # sample standard deviation 
    n = len(x)
    m = np.median(x)
    return  round(m - (t * s / np.sqrt(n)),4)

def CI_upper_median(x):
    """Computes 95% Confidence Interval Upper Bound of the Median"""
    alpha = 0.05                       # significance level = 5%
    degfree = len(x) - 1                  # degress of freedom
    t = stats.t.ppf(1 - alpha/2, degfree)   # 95% confidence t-score 
    s = np.std(x)          # sample standard deviation 
    n = len(x)
    m = np.median(x)
    return  round(m + (t * s / np.sqrt(n)),4)
    
# To calculate estimated cumulative distribution functions if required
    
def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    # Number of data points: n
    n = len(data)
    # x-data for the ECDF: x
    x = np.sort(data)
    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n
    return x, y

#### Load in the Datasets

Now we are ready to load in the datasets from the processing steps.

In [ ]:
# Load in clean data

df = pd.read_csv('{path_to_clean_data_from_data_processing/clean_data_file.csv}')

#### Quick sense-check

In [ ]:
# Quickly check the data loaded in

print(df.shape)
df.head()

#### Now in a single line we can compute all the major differences between the GIBLU and non-GIBLU patients

Using the .agg function we can very quickly get a good idea about the differences between the two cohorts

In [ ]:
# Create a full aggregated table of results

df_agg = df.groupby('GIBLU').agg(['mean', CI_lower, CI_upper, 'median', CI_lower_median, CI_upper_median, IQR_lower, IQR_upper,'sum','count',std_lower, std_upper])
df_agg.head()

#### Building a Results table

This next part is not the most elegant way to tabulate the results and associated statistics but it is the easiest to understand way to do it (ie. no for loops required). I suggest you start by doing it this way (by wrote) and then try to write the same out in a series of loops. Then finally in a single loop.

In [ ]:
# Create Unique Dataframes for each metric

mean = df_agg.iloc[:, df_agg.columns.get_level_values(1)=='mean'].T
mean_CIL = df_agg.iloc[:, df_agg.columns.get_level_values(1)=='CI_lower'].T
mean_CIU = df_agg.iloc[:, df_agg.columns.get_level_values(1)=='CI_upper'].T
median = df_agg.iloc[:, df_agg.columns.get_level_values(1)=='median'].T
CI_lower_median = df_agg.iloc[:, df_agg.columns.get_level_values(1)=='CI_lower_median'].T
CI_upper_median = df_agg.iloc[:, df_agg.columns.get_level_values(1)=='CI_upper_median'].T
IQR_lower = df_agg.iloc[:, df_agg.columns.get_level_values(1)=='IQR_lower'].T
IQR_upper = df_agg.iloc[:, df_agg.columns.get_level_values(1)=='IQR_upper'].T
sums = df_agg.iloc[:, df_agg.columns.get_level_values(1)=='sum'].T
counts = df_agg.iloc[:, df_agg.columns.get_level_values(1)=='count'].T
std_lower = df_agg.iloc[:, df_agg.columns.get_level_values(1)=='std_lower'].T
std_upper = df_agg.iloc[:, df_agg.columns.get_level_values(1)=='std_upper'].T

# Re-label the columns to make it human readable

mean.columns = ['NIBLU Mean', 'GIBLU Mean']
mean_CIL.columns = ['NIBLU Mean 95%CI Lower', 'GIBLU Mean 95%CI Lower']
mean_CIU.columns = ['NIBLU Mean 95%CI Upper', 'GIBLU Mean 95%CI Upper']
median.columns = ['NIBLU Median', 'GIBLU Median']
CI_lower_median.columns = ['NIBLU Median 95%CI Lower', 'GIBLU Median 95%CI Lower']
CI_upper_median.columns = ['NIBLU Median 95%CI Upper', 'GIBLU Median 95%CI Upper']
IQR_lower.columns = ['NIBLU IQR Lower', 'GIBLU IQR Lower']
IQR_upper.columns = ['NIBLU IQR Upper', 'GIBLU IQR Upper']
IQR_upper.columns = ['NIBLU IQR Upper', 'GIBLU IQR Upper']
sums.columns = ['NIBLU sum', 'GIBLU sum']
counts.columns = ['NIBLU count', 'GIBLU count']
std_lower.columns = ['NIBLU std_lower', 'GIBLU std_lower']
std_upper.columns = ['NIBLU std_upper', 'GIBLU std_upper']

# Then reset the index

means = mean.reset_index(0).reset_index(drop=True)
mean_CILs = mean_CIL.reset_index(0).reset_index(drop=True)
mean_CIUs = mean_CIU.reset_index(0).reset_index(drop=True)
medians = median.reset_index(0).reset_index(drop=True)
median_CILs = CI_lower_median.reset_index(0).reset_index(drop=True)
median_CIUs = CI_upper_median.reset_index(0).reset_index(drop=True)
IQR_lowers = IQR_lower.reset_index(0).reset_index(drop=True)
IQR_uppers = IQR_upper.reset_index(0).reset_index(drop=True)
sums = sums.reset_index(0).reset_index(drop=True)
counts = counts.reset_index(0).reset_index(drop=True)
std_lower = std_lower.reset_index(0).reset_index(drop=True)
std_upper = std_upper.reset_index(0).reset_index(drop=True)

# Then join them all together into a single dataframe

join1 = pd.merge(means, mean_CILs, on='level_0')
join2 = pd.merge(join1, mean_CIUs, on='level_0')
join3 = pd.merge(join2, medians, on='level_0')
join4 = pd.merge(join3, median_CILs, on='level_0')
join5 = pd.merge(join4, median_CIUs, on='level_0')
join6 = pd.merge(join5, IQR_lowers, on='level_0')
join7 = pd.merge(join6, IQR_uppers, on='level_0')
join8 = pd.merge(join7, sums, on='level_0')
join9 = pd.merge(join8, counts, on='level_0')
join10 = pd.merge(join9, std_lower, on='level_0')
join11 = pd.merge(join10, std_upper, on='level_0')

#### You now have a single table of results

This is your master results table. At this point to calculate the statistics it is easiest to understand if we split the tables into two and then calculate the stats

In [ ]:
# Split the two cohorts into GIBLU and not intestinal bleed unit - NIBLU ;D

GIBLU = df_cohort[df_cohort['GIBLU'] == 1]
NIBLU = df_cohort[df_cohort['GIBLU'] == 0]

# First test that the mannwhitney U test is working and you can extract the result successfully

a = stats.mannwhitneyu(GIBLU['age'],NIBLU['age'])
a_p = a[1]
print(a_p)

# Then build a Chi2*2 dataframe table to calculate this

b1 = GIBLU.loc[:,'sex'].sum()
b2 = NIBLU.loc[:,'sex'].sum()
b3 = GIBLU.loc[:,'sex'].count()-GIBLU.loc[:,'sex'].sum()
b4 = NIBLU.loc[:,'sex'].count()-NIBLU.loc[:,'sex'].sum()
contingencyb = pd.DataFrame(np.array([[b1, b2], [b3, b4]]), columns=['GIBLU','NIBLU'])
print(contingencyb.head())
b = stat,p,dof,expected = chi2_contingency(contingencyb)
b_p = b[1]
print(b_p)

#### Once you know this is working you can go through the rest 

The rest of the stats will be either MannWhitneyU tests for continuous or Chi2 for categorical data. For the sake of simplicity these are all programmed explicitly here but as with the above they can more efficiently be performed in a loop or even better using a function. Again start with this and then work up to the latter. I have labelled them all with letters for each covariate to keep it simple (which works fine for a small dataset like this) and '_p' to denote a P value.

In [ ]:
# Then add charlson scores

c = stats.mannwhitneyu(GIBLU['age_adj_quan_wt_charlson_icd10_quan'],NIBLU['age_adj_quan_wt_charlson_icd10_quan'])
c_p = c[1]
print(c_p)

# And 10 year survival estimates

d = stats.mannwhitneyu(GIBLU['survival_10yr'],NIBLU['survival_10yr'])
d_p = d[1]
print(d_p)

# Then hfrs

e = stats.mannwhitneyu(GIBLU['hfrs'],NIBLU['hfrs'])
e_p = e[1]
print(e_p)

# Then rescope rate

f = stats.mannwhitneyu(GIBLU['rescoped'],NIBLU['rescoped'])
f_p = f[1]
print(f_p)

# Then length of stay

g = stats.mannwhitneyu(GIBLU['length_of_stay'],NIBLU['length_of_stay'])
g_p = h[1]
print(h_p)

# Then readmission within 7 days

h = stats.mannwhitneyu(GIBLU['readmitted_within_7_days'],NIBLU['readmitted_within_7_days'])
h_p = g[1]
print(g_p)

# And finally 30 day mortality

i1 = GIBLU.loc[:,'30_day_mortality'].sum()
i2 = NIBLU.loc[:,'30_day_mortality'].sum()
i3 = GIBLU.loc[:,'30_day_mortality'].count()-GIBLU.loc[:,'30_day_mortality'].sum()
i4 = NIBLU.loc[:,'30_day_mortality'].count()-NIBLU.loc[:,'30_day_mortality'].sum()
contingencyi = pd.DataFrame(np.array([[i1, i2], [i3, i4]]), columns=['GIBLU','NIBLU'])
print(contingencyi.head())
i = stat,p,dof,expected = chi2_contingency(contingencyi)
i_p = i[1]
print(i_p)

#### Now we can bolt this on to our earlier joined dataframe

We can just bolt the p_values on. This is obviously a bit of a clumsy way to do it but it makes it much easier to understand thus why we have written it out so verbosely here.

In [ ]:
# Append the p values to the joined statistical dataframe

join11['p'] = np.array([a_p,b_p,c_p,d_p,e_p,f_p,g_p,h_p,i_p], dtype=object)

# Then switch the index to match the correct columns

join12 = join11.set_index('level_0')
join12

#### Finally, if we want to we can calculate the differences between variables here

This is sometimes required for papers and journals and is now obviously very easy to do

In [ ]:
# To calculate the differences by wrote you can do

join12['Median Differences'] = join12['NIBLU Median'] - join12['GIBLU Median']
join12['Median Lower CI Differences'] = join12['NIBLU Median 95%CI Lower'] - join12['GIBLU Median 95%CI Lower']
join12['Median Upper CI Differences'] = join12['NIBLU Median 95%CI Upper'] - join12['GIBLU Median 95%CI Upper']
join12['Mean Differences'] = join12['NIBLU Mean'] - join12['GIBLU Mean']
join12['Mean Lower CI Differences'] = join12['NIBLU Mean 95%CI Lower'] - join12['GIBLU Mean 95%CI Lower']
join12['Mean Upper CI Differences'] = join12['NIBLU Mean 95%CI Upper'] - join12['GIBLU Mean 95%CI Upper']
join12['Mean Differences'] = join12['Mean Differences']*100
join12['Mean Lower CI Differences'] = join12['Mean Lower CI Differences']*100
join12['Mean Upper CI Differences'] = join12['Mean Upper CI Differences']*100

#### Finally to perform logstic regression and build a model we can use the statsmodels.api 

I will add this in later as the above is probably enough for newcomers. Once we start adding in statistical models some people will be put off. 

If you want to have this however, please get in contact. If there is demand I will just append it to the bottom of this script. If not I will probably keep it simple for now.